In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import re

In [2]:
# 목록 URL
def find_url(page_idx):
    url = "https://www.smc.seoul.kr/info/billSearchDetail.do"
    url = f"{url}?checkValue=byhomepage&hidNo=&propTypeCd=&billNo=&billTypeCd=1"
    url = f"{url}&billNum=&pageIndex={page_idx}&menuId=006002003&boardTypeId=&schCsel=&pageSize="
    url = f"{url}&url=%2FbillSearchDetail.do&startDate=2022-07-01&endDate=2023-03-30"
    url = f"{url}&billNm=&generationNum=011&propMemberTypeCdName=&propType=01&committee=0"
    url = f"{url}&committeeTypeCd=1&billType=0&genmeetResult=0&propTypeCode=01&propMemberTypeCd=10"
    url = f"{url}&tempPropMemberTypeCd=10&searchBillNm="
    return url

In [3]:
page_idx = 35
print(find_url(page_idx))

https://www.smc.seoul.kr/info/billSearchDetail.do?checkValue=byhomepage&hidNo=&propTypeCd=&billNo=&billTypeCd=1&billNum=&pageIndex=35&menuId=006002003&boardTypeId=&schCsel=&pageSize=&url=%2FbillSearchDetail.do&startDate=2022-07-01&endDate=2023-03-30&billNm=&generationNum=011&propMemberTypeCdName=&propType=01&committee=0&committeeTypeCd=1&billType=0&genmeetResult=0&propTypeCode=01&propMemberTypeCd=10&tempPropMemberTypeCd=10&searchBillNm=


In [4]:
# 상세페이지 URL
def sub_find_url(page_idx, billNo):
    url = "https://www.smc.seoul.kr/info/billRead.do?checkValue=byhomepage"
    url = f"{url}&hidNo=&propTypeCd=01&billNo={billNo}&billTypeCd=1&billNum=1"
    url = f"{url}&pageIndex={page_idx}&menuId=006002003&boardTypeId=&schCsel="
    url = f"{url}&pageSize=&url=%2FbillSearchDetail.do&startDate=2022-07-01&endDate=2023-03-30"
    url = f"{url}&billNm=&generationNum=011&propMemberTypeCdName=&propType=01&committee=0"
    url = f"{url}&committeeTypeCd=1&billType=0&genmeetResult=0&propTypeCode=01&propMemberTypeCd=10"
    url = f"{url}&tempPropMemberTypeCd=10&searchBillNm="
    return url

In [5]:
print(sub_find_url(page_idx=1, billNo="00700"))

https://www.smc.seoul.kr/info/billRead.do?checkValue=byhomepage&hidNo=&propTypeCd=01&billNo=00700&billTypeCd=1&billNum=1&pageIndex=1&menuId=006002003&boardTypeId=&schCsel=&pageSize=&url=%2FbillSearchDetail.do&startDate=2022-07-01&endDate=2023-03-30&billNm=&generationNum=011&propMemberTypeCdName=&propType=01&committee=0&committeeTypeCd=1&billType=0&genmeetResult=0&propTypeCode=01&propMemberTypeCd=10&tempPropMemberTypeCd=10&searchBillNm=


In [6]:
page_list = []
for i in range(1, 36):
    url = find_url(page_idx)
    temp = pd.read_html(url)[0]
    page_list.append(temp)

In [7]:
from datetime import datetime

today = datetime.today().strftime("%Y-%m-%d")
today

'2023-04-05'

In [8]:
file_name = f"서울시의안목록-{today}.csv"
file_name

'서울시의안목록-2023-04-05.csv'

In [9]:
df = pd.concat(page_list)
df.to_csv("의안목록.csv", index=False)

In [10]:
pd.read_csv("의안목록.csv")

,번호,의안명,제안자,제안일
0,11-00048,서울특별시 민원업무 수행 공무원 등의 보호 및 지원에 관한 조례안 소관위원회 : ...,의원,2022-07-22
1,11-00012,서울특별시 주거 기본 조례 일부개정조례안 소관위원회 : 주택공간,의원,2022-07-06
2,11-00011,서울특별시 쪽방주민의 복지 및 생활안정지원에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06
3,11-00010,서울특별시 공공보건의료에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06
4,11-00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안 소관위원회 : 보건복지,의원,2022-07-06
...,...,...,...,...
275,11-00010,서울특별시 공공보건의료에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06
276,11-00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안 소관위원회 : 보건복지,의원,2022-07-06
277,11-00008,서울특별시 스토킹범죄 예방 및 피해지원에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06
278,11-00003,서울특별시 미디어재단 티비에스(tbs) 설립 및 운영에 관한 조례 폐지조례안 소관...,의원,2022-07-04


In [11]:
def find_sub_page(billNo):
    sub_url = sub_find_url(page_idx=1, billNo=billNo)
    response = requests.get(sub_url)
    html = bs(response.text)
    sub_text = " ".join([row.text for row in html.select("div.row")])
    remove_text = "\n상임위원회 선택하시면 홈페이지를 확인하실 수 있습니다.\n\n상임위원회 창닫기\n\n \n\n페이스북\n페이스북\n\n\n트위터\n트위터\n\n\n카카오\n카카오\n\n\n네이버블로그\n네이버블로그\n\n\n다음블로그\n다음블로그\n\n\nURL복사\nURL복사\n\n \n\n\n서울특별시의회 홈페이지는 PC, 태블릿, 스마트폰에 최적화된 반응형 웹 기술로 구축되었습니다.\r\n원활한 서비스 이용을 위해 IE11이나 크롬브라우저 사용을 권장합니다.\r\n          \n\ndownload IE11\ndownload chrome\n\n\n\n브라우저 권장 창닫기\n\n\n \n\n\n맨 위로 가기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSNS 공유\n트위터\n카카오스토리\n네이버블로그\nURL\nPRINT\nRSS\nmore\n\n\n트위터\n카카오스토리\n네이버블로그\nURL\n\n\n\n네이버블로그\n트위터\n카카오스토리\nURL\n\n\n네이버블로그\n트위터\n카카오스토리\nURL\n\n\n\n\n\n\n\n\n\n\n\n\n"
    sub_text = sub_text.replace(remove_text, " ")
    return sub_text

In [12]:
billNo = "00699"
find_sub_page(billNo)

' 서울특별시 재난 예보·경보시스템 구축 및 운영에 관한 조례 일부개정조례안\n\n\n\n의안번호11-00699\n\n\n의안종류조례안\n\n\n의안구분원안\n\n\n\n\n\n\n제안일2023-03-29\n\n\n제안자\n\r\n\t                                        \t\r\n\t                                        \t\t\r\n\t                                        \t\t\t의원\r\n\t                                        \t\t\r\n\t                                        \t\t\r\n\t\t                                        \t\r\n\t\t                                        \t\r\n\t                                        \t\t\r\n\t                                        \t\t\r\n\t                                        \t\r\n\t                                        \n\n\n\n\n\n\n발의구분\n\r\n\t\t                                        \t\r\n\t\t                                        \t\t\r\n\t\t                                        \t\t1인발의\r\n\t\t                                        \t\t\r\n\t\t                                        \t\t\r\n\t\t                                       

In [13]:
df["billNo"] = df["번호"].str.split("-", expand=True)[1]

In [14]:
from tqdm import tqdm
tqdm.pandas()

df["내용"] = df["billNo"].progress_map(find_sub_page)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 280/280 [00:34<00:00,  8.09it/s]


In [15]:
df

,번호,의안명,제안자,제안일,billNo,내용
0,11-00048,서울특별시 민원업무 수행 공무원 등의 보호 및 지원에 관한 조례안 소관위원회 : ...,의원,2022-07-22,00048,서울특별시 민원업무 수행 공무원 등의 보호 및 지원에 관한 조례안\n\n\n\n의...
1,11-00012,서울특별시 주거 기본 조례 일부개정조례안 소관위원회 : 주택공간,의원,2022-07-06,00012,서울특별시 주거 기본 조례 일부개정조례안\n\n\n\n의안번호11-00012\n\...
2,11-00011,서울특별시 쪽방주민의 복지 및 생활안정지원에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,00011,서울특별시 쪽방주민의 복지 및 생활안정지원에 관한 조례안\n\n\n\n의안번호11...
3,11-00010,서울특별시 공공보건의료에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,00010,서울특별시 공공보건의료에 관한 조례안\n\n\n\n의안번호11-00010\n\n\...
4,11-00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안 소관위원회 : 보건복지,의원,2022-07-06,00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안\n\n\n\n의안번호11-...
...,...,...,...,...,...,...
3,11-00010,서울특별시 공공보건의료에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,00010,서울특별시 공공보건의료에 관한 조례안\n\n\n\n의안번호11-00010\n\n\...
4,11-00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안 소관위원회 : 보건복지,의원,2022-07-06,00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안\n\n\n\n의안번호11-...
5,11-00008,서울특별시 스토킹범죄 예방 및 피해지원에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,00008,서울특별시 스토킹범죄 예방 및 피해지원에 관한 조례안\n\n\n\n의안번호11-0...
6,11-00003,서울특별시 미디어재단 티비에스(tbs) 설립 및 운영에 관한 조례 폐지조례안 소관...,의원,2022-07-04,00003,서울특별시 미디어재단 티비에스(tbs) 설립 및 운영에 관한 조례 폐지조례안\n\...


In [16]:
df.to_csv("서울시의안목록.csv", index=False)

df = pd.read_csv("서울시의안목록.csv")
df

,번호,의안명,제안자,제안일,billNo,내용
0,11-00048,서울특별시 민원업무 수행 공무원 등의 보호 및 지원에 관한 조례안 소관위원회 : ...,의원,2022-07-22,48,서울특별시 민원업무 수행 공무원 등의 보호 및 지원에 관한 조례안\n\n\n\n의...
1,11-00012,서울특별시 주거 기본 조례 일부개정조례안 소관위원회 : 주택공간,의원,2022-07-06,12,서울특별시 주거 기본 조례 일부개정조례안\n\n\n\n의안번호11-00012\n\...
2,11-00011,서울특별시 쪽방주민의 복지 및 생활안정지원에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,11,서울특별시 쪽방주민의 복지 및 생활안정지원에 관한 조례안\n\n\n\n의안번호11...
3,11-00010,서울특별시 공공보건의료에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,10,서울특별시 공공보건의료에 관한 조례안\n\n\n\n의안번호11-00010\n\n\...
4,11-00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안 소관위원회 : 보건복지,의원,2022-07-06,9,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안\n\n\n\n의안번호11-...
...,...,...,...,...,...,...
275,11-00010,서울특별시 공공보건의료에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,10,서울특별시 공공보건의료에 관한 조례안\n\n\n\n의안번호11-00010\n\n\...
276,11-00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안 소관위원회 : 보건복지,의원,2022-07-06,9,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안\n\n\n\n의안번호11-...
277,11-00008,서울특별시 스토킹범죄 예방 및 피해지원에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,8,서울특별시 스토킹범죄 예방 및 피해지원에 관한 조례안\n\n\n\n의안번호11-0...
278,11-00003,서울특별시 미디어재단 티비에스(tbs) 설립 및 운영에 관한 조례 폐지조례안 소관...,의원,2022-07-04,3,서울특별시 미디어재단 티비에스(tbs) 설립 및 운영에 관한 조례 폐지조례안\n\...


In [17]:
df["발의의원"] = df["내용"].map(lambda x : x.split("발의구분")[1].split("소관위원회")[0])
df["발의의원"] = df["발의의원"].map(lambda x : re.sub(r'[\xa0\r\n\t]+', " ", x))
df["발의의원"] = df["발의의원"].map(lambda x : x.split("심사일처리결과")[0])

In [18]:
df["발의구분"] = df["내용"].map(lambda x : x.split("발의구분")[1].split("발의")[0])
df["발의구분"] = df["발의구분"].map(lambda x : re.sub(r'[\xa0\r\n\t]+', " ", x))
df["발의구분"] = df["발의구분"].map(lambda x : re.sub(r'[\s]+', "", x))
df.loc[0, "발의구분"]

'공동'

In [19]:
df.loc[0, "발의의원"]

'                                                                                                                                                                     공동발의                                                                                                                            공동발의의원  1                                      최호정(崔好廷) (국민의힘)                                      2                                      강석주(姜錫周) (국민의힘)                                      3                                      경기문(景琦文) (국민의힘)                                      4                                      고광민(高光敏) (국민의힘)                                      5                                      곽향기(郭香基) (국민의힘)                                      6                                      구미경(具美慶) (국민의힘)                                      7                                      김경훈(金熲勳) (국민의힘)                                      8                                      김규남(金奎男) (국민의힘)

In [20]:
df

,번호,의안명,제안자,제안일,billNo,내용,발의의원,발의구분
0,11-00048,서울특별시 민원업무 수행 공무원 등의 보호 및 지원에 관한 조례안 소관위원회 : ...,의원,2022-07-22,48,서울특별시 민원업무 수행 공무원 등의 보호 및 지원에 관한 조례안\n\n\n\n의...,...,공동
1,11-00012,서울특별시 주거 기본 조례 일부개정조례안 소관위원회 : 주택공간,의원,2022-07-06,12,서울특별시 주거 기본 조례 일부개정조례안\n\n\n\n의안번호11-00012\n\...,...,공동
2,11-00011,서울특별시 쪽방주민의 복지 및 생활안정지원에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,11,서울특별시 쪽방주민의 복지 및 생활안정지원에 관한 조례안\n\n\n\n의안번호11...,...,공동
3,11-00010,서울특별시 공공보건의료에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,10,서울특별시 공공보건의료에 관한 조례안\n\n\n\n의안번호11-00010\n\n\...,...,공동
4,11-00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안 소관위원회 : 보건복지,의원,2022-07-06,9,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안\n\n\n\n의안번호11-...,...,공동
...,...,...,...,...,...,...,...,...
275,11-00010,서울특별시 공공보건의료에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,10,서울특별시 공공보건의료에 관한 조례안\n\n\n\n의안번호11-00010\n\n\...,...,공동
276,11-00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안 소관위원회 : 보건복지,의원,2022-07-06,9,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안\n\n\n\n의안번호11-...,...,공동
277,11-00008,서울특별시 스토킹범죄 예방 및 피해지원에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,8,서울특별시 스토킹범죄 예방 및 피해지원에 관한 조례안\n\n\n\n의안번호11-0...,...,공동
278,11-00003,서울특별시 미디어재단 티비에스(tbs) 설립 및 운영에 관한 조례 폐지조례안 소관...,의원,2022-07-04,3,서울특별시 미디어재단 티비에스(tbs) 설립 및 운영에 관한 조례 폐지조례안\n\...,...,공동


In [21]:
df.to_csv("서울시의안목록.csv", index=False)

df = pd.read_csv("서울시의안목록.csv")
df

,번호,의안명,제안자,제안일,billNo,내용,발의의원,발의구분
0,11-00048,서울특별시 민원업무 수행 공무원 등의 보호 및 지원에 관한 조례안 소관위원회 : ...,의원,2022-07-22,48,서울특별시 민원업무 수행 공무원 등의 보호 및 지원에 관한 조례안\n\n\n\n의...,...,공동
1,11-00012,서울특별시 주거 기본 조례 일부개정조례안 소관위원회 : 주택공간,의원,2022-07-06,12,서울특별시 주거 기본 조례 일부개정조례안\n\n\n\n의안번호11-00012\n\...,...,공동
2,11-00011,서울특별시 쪽방주민의 복지 및 생활안정지원에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,11,서울특별시 쪽방주민의 복지 및 생활안정지원에 관한 조례안\n\n\n\n의안번호11...,...,공동
3,11-00010,서울특별시 공공보건의료에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,10,서울특별시 공공보건의료에 관한 조례안\n\n\n\n의안번호11-00010\n\n\...,...,공동
4,11-00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안 소관위원회 : 보건복지,의원,2022-07-06,9,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안\n\n\n\n의안번호11-...,...,공동
...,...,...,...,...,...,...,...,...
275,11-00010,서울특별시 공공보건의료에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,10,서울특별시 공공보건의료에 관한 조례안\n\n\n\n의안번호11-00010\n\n\...,...,공동
276,11-00009,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안 소관위원회 : 보건복지,의원,2022-07-06,9,서울특별시 주민생활안정 지원에 관한 조례 일부개정조례안\n\n\n\n의안번호11-...,...,공동
277,11-00008,서울특별시 스토킹범죄 예방 및 피해지원에 관한 조례안 소관위원회 : 보건복지,의원,2022-07-06,8,서울특별시 스토킹범죄 예방 및 피해지원에 관한 조례안\n\n\n\n의안번호11-0...,...,공동
278,11-00003,서울특별시 미디어재단 티비에스(tbs) 설립 및 운영에 관한 조례 폐지조례안 소관...,의원,2022-07-04,3,서울특별시 미디어재단 티비에스(tbs) 설립 및 운영에 관한 조례 폐지조례안\n\...,...,공동
